In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('DataSet/Train_Dataset.csv')
df_test = pd.read_csv('DataSet/Test_Dataset.csv')

In [3]:
# Check for the No. of Data Available of an Country
df['Country'].value_counts().tail(11)

Ghana                    15
Tuvalu                    1
Cook Islands              1
Marshall Islands          1
Monaco                    1
Palau                     1
Niue                      1
San Marino                1
Nauru                     1
Saint Kitts and Nevis     1
Dominica                  1
Name: Country, dtype: int64

In [4]:
# Removing the Countries with less than 15 Yrs of Data
country = df.iloc[:,1].value_counts().index[-10:]
for i in country :
    df.drop(df[df['Country'] == i].index, inplace=True, axis = 0)

In [5]:
# Removing the Features Not Present in Test Dataset
Y_train = df['Power Level']
df.drop('Power Level', axis = 1,inplace = True)
# As all these features are Not Present in Test Dataset
df.drop('Unnamed: 0', axis = 1, inplace = True)
df.drop('Substance Abuse', axis = 1, inplace = True)
df.drop('Educational Investment', axis = 1, inplace = True)

In [6]:
# Changing Object to Category
# Train Dataset
df['Country'] = df['Country'].astype('category')
df['Country'] = df['Country'].cat.codes
df['Status'] = df['Status'].astype('category')
df['Status'] = df['Status'].cat.codes
# Test Dataset
df_test['Country'] = df_test['Country'].astype('category')
df_test['Country'] = df_test['Country'].cat.codes
df_test['Status'] = df_test['Status'].astype('category')
df_test['Status'] = df_test['Status'].cat.codes

In [7]:
# Filling the NaN Values
# Train Dataset
for i in range(0,183) :
    cat = df[df['Country'] == i]
    for j in range(3,19) :
        if math.isnan(np.mean(cat.iloc[:,j])) == True :
            cat.iloc[:,j] = cat.iloc[:,j].fillna(np.mean(df.iloc[:,j]))
        else :  
            cat.iloc[:,j] = cat.iloc[:,j].fillna(np.mean(cat.iloc[:,j]))
    df[df['Country'] == i] = cat
# Test Dataset
for k in range(3,19) :
    df_test.iloc[:,k] = df_test.iloc[:,k].fillna(np.mean(df_test.iloc[:,k]))

/var/folders/wc/rkrv7ck92zd4f_3qgk8q2gn00000gn/T/ipykernel_8286/3307623477.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat.iloc[:,j] = cat.iloc[:,j].fillna(np.mean(cat.iloc[:,j]))
/var/folders/wc/rkrv7ck92zd4f_3qgk8q2gn00000gn/T/ipykernel_8286/3307623477.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cat.iloc[:,j] = cat.iloc[:,j].fillna(np.mean(df.iloc[:,j]))


In [8]:
# Z-Score Normalization
# Train Dataset
for i in range(3,19) :
    df.iloc[:,i] = (df.iloc[:,i] - np.mean(df.iloc[:,i]))/np.std(df.iloc[:,i])
# Test Dataset
for i in range(3,19) :
    df_test.iloc[:,i] = (df_test.iloc[:,i] - np.mean(df_test.iloc[:,i]))/np.std(df_test.iloc[:,i])

In [9]:
# Linear Regression
reg = LinearRegression()
reg.fit(df,Y_train)

LinearRegression()

In [10]:
# Saving the Model
# import pickle
# pickle.dump(reg, open('model.pkl','wb'))

# Loading the Model
# model = pickle.load(open('model.pkl','rb'))

# Predicting the Power Level
Y_test = reg.predict(df_test)

# Saving the Predicted Power Level
df_output = {'Country ID': np.arange(0,183), 'Predicted Power Level': Y_test}
df_output = pd.DataFrame(df_output)
df_output.to_csv('DataSet/Predicted Output.csv')

In [11]:
# Evaluating the Predicted Output with the Expected Output
from sklearn.metrics import mean_squared_error

# Load the Expected Output.csv and Predicted Output.csv File
X = pd.read_csv('DataSet/Predicted Output.csv')
Y = pd.read_csv('DataSet/Expected Output.csv')

X = X['Predicted Power Level']
Y = Y['Power Level']

print(mean_squared_error(X, Y))

22.141372017302608
